In [4]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
import numpy as np
import time
import warnings

In [7]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

In [6]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import shap

# Load Data

In [23]:
df = pd.read_csv("../../data/AmesHousing.csv")
df.head()
df = df.select_dtypes(include="number")
df.dropna(inplace=True)
df["random_feature"] = np.random.random(df.shape[0])
df["Overall Qual Noise"] = df["Overall Qual"] + (df["random_feature"] * 0.0001)
# Split the data
test_size = 0.3
raw_X_train, X_test, raw_y_train, y_test = train_test_split(df.drop("SalePrice", axis=1), df["SalePrice"], test_size=test_size, random_state=42)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(raw_X_train, raw_y_train, test_size=test_size, random_state=42)

# Feature Selection

In [26]:
from sklearn.metrics import mean_absolute_percentage_error

In [46]:
# define your own objective function, make sure the function receives four parameters,
#  fit your model and return the objective value !
def objective_function_topass(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    P = mean_absolute_percentage_error(y_test, model.predict(X_test))
    return P

In [47]:
# import an algorithm !
from zoofs import ParticleSwarmOptimization

# create object of algorithm
algo_object = ParticleSwarmOptimization(
    objective_function_topass, n_iteration=20, population_size=5, minimize=True
)

In [48]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(n_estimators=50, n_jobs=-1)

In [49]:
from category_encoders import MEstimateEncoder
from sklearn.pipeline import Pipeline

In [50]:
%%time
# fit the algorithm
algo_object.fit(lgb_model, X_train, y_train, X_valid, y_valid, verbose=False)

CPU times: user 9.8 s, sys: 18.3 s, total: 28.1 s
Wall time: 18.9 s


['Order',
 'PID',
 'MS SubClass',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 '1st Flr SF',
 'Gr Liv Area',
 'Bsmt Half Bath',
 'Kitchen AbvGr',
 'Fireplaces',
 'Garage Cars',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 'Pool Area',
 'Mo Sold',
 'Yr Sold',
 'random_feature',
 'Overall Qual Noise']

In [51]:
# plot your results
algo_object.plot_history()

In [52]:
algo_object.best_score

0.08577454064215163

In [53]:
final_cols = algo_object.best_feature_list

In [54]:
final_cols

['Order',
 'PID',
 'MS SubClass',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 '1st Flr SF',
 'Gr Liv Area',
 'Bsmt Half Bath',
 'Kitchen AbvGr',
 'Fireplaces',
 'Garage Cars',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 'Pool Area',
 'Mo Sold',
 'Yr Sold',
 'random_feature',
 'Overall Qual Noise']

In [83]:
%%time
full_lgb_model = lgb.LGBMRegressor(n_estimators=100, n_jobs=-1)
full_lgb_model.fit(
    raw_X_train[final_cols], raw_y_train
)

CPU times: user 224 ms, sys: 460 ms, total: 684 ms
Wall time: 516 ms


LGBMRegressor()

In [84]:
y_pred = full_lgb_model.predict(X_test[final_cols])

In [85]:
mean_absolute_percentage_error(y_test, y_pred)

0.1021026084734843

In [86]:
import numpy as np

In [69]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Creating the histograms
trace1 = go.Histogram(x=y_test, opacity=0.75, name="Actual")
trace2 = go.Histogram(x=y_pred, opacity=0.75, name="Predicted")

data = [trace1, trace2]

layout = go.Layout(
    title="Overlay of Actual and Predicted Values",
    xaxis=dict(title="Values"),
    yaxis=dict(title="Count"),
    barmode="overlay",
)

fig = go.Figure(data=data, layout=layout)

# Show the plot
pyo.iplot(fig)